## Model Creation and Training with PyTorch

This notebook demonstrates:
- Loading FashionMNIST dataset
- Creating DataLoaders
- Defining a neural network
- Training and evaluating the model
- Saving the trained model


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Loading the Dataset

We use the FashionMNIST dataset: 28x28 grayscale images of 10 clothing classes, the code is copied from the notebook in folder `[00]_dataset_and_dataloader`



In [2]:
# ----- NECESSARY CODE COMING FROM [00]_dataset_and_dataloader -------
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
# ----- NECESSARY CODE COMING FROM [00]_dataset_and_dataloader -------

## Choosing Device

We will use GPU if available, otherwise CPU.


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device

'cuda'

## Defining the Neural Network

- Flatten input images
- Two hidden layers with ReLU
- Output layer with 10 classes


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Loss Function and Optimizer

- CrossEntropyLoss for multi-class classification  
- SGD optimizer with learning rate 0.001


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## Training Function

For each batch:
1. Move inputs and labels to device
2. Forward pass
3. Compute loss
4. Backward pass
5. Update weights


In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss_val, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")

## Testing Function

Evaluate model on the test set without updating weights.


In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Training the Model

We train for 5 epochs and test after each epoch.


In [8]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Training completed!")

Epoch 1
-------------------------------
loss: 2.307945  [   64/60000]
loss: 2.301825  [ 6464/60000]
loss: 2.277954  [12864/60000]
loss: 2.266788  [19264/60000]
loss: 2.240473  [25664/60000]
loss: 2.210480  [32064/60000]
loss: 2.225921  [38464/60000]
loss: 2.184905  [44864/60000]
loss: 2.194077  [51264/60000]
loss: 2.142201  [57664/60000]
Test Error: 
 Accuracy: 33.6%, Avg loss: 2.145528 

Epoch 2
-------------------------------
loss: 2.163489  [   64/60000]
loss: 2.156240  [ 6464/60000]
loss: 2.093937  [12864/60000]
loss: 2.106387  [19264/60000]
loss: 2.046939  [25664/60000]
loss: 1.987151  [32064/60000]
loss: 2.023199  [38464/60000]
loss: 1.936327  [44864/60000]
loss: 1.958634  [51264/60000]
loss: 1.866896  [57664/60000]
Test Error: 
 Accuracy: 56.7%, Avg loss: 1.869795 

Epoch 3
-------------------------------
loss: 1.913558  [   64/60000]
loss: 1.876840  [ 6464/60000]
loss: 1.759488  [12864/60000]
loss: 1.795758  [19264/60000]
loss: 1.680484  [25664/60000]
loss: 1.637649  [32064/600

## Saving the Trained Model
Save the model state for later use.

In [9]:
torch.save(model.state_dict(), "./model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In the next notebook we will see how to load the saved model and use it for inference.